
# Pneumonia Detection with MobileNetV2 (Lightweight Pipeline)
This notebook uses MobileNetV2 and Keras data generators for efficient pneumonia detection.

In [1]:
# 1. Import Required Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
# 2. Data Generators for MobileNetV2 (224x224 RGB)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'chest_xray/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    color_mode='rgb',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    'chest_xray/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    color_mode='rgb',
    subset='validation'
)

Found 2807 images belonging to 2 classes.
Found 700 images belonging to 2 classes.


In [ ]:
# 3. Define and Compile the MobileNetV2 Model
# (Original custom CNN model is commented out for reference)
'''
model = Sequential([
    Conv2D(...),
    MaxPooling2D(...),
    # ... more layers ...
    Dense(1, activation='sigmoid')
])
'''
        
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base for initial training

inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)  # Optional: helps regularization
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [4]:
# 4. Train the Model and Save the Best Version
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('best_pneumonia_model.h5', save_best_only=True)
]
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=callbacks
)

print('Model training complete. Best model saved as best_pneumonia_model.h5')

c:\Users\ASUS\.conda\envs\GPUT\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 643ms/step - accuracy: 0.7092 - loss: 0.5543

88/88 ━━━━━━━━━━━━━━━━━━━━ 77s 839ms/step - accuracy: 0.7103 - loss: 0.5526 - val_accuracy: 0.8986 - val_loss: 0.2612
Epoch 2/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 616ms/step - accuracy: 0.9031 - loss: 0.2381

88/88 ━━━━━━━━━━━━━━━━━━━━ 69s 782ms/step - accuracy: 0.9031 - loss: 0.2381 - val_accuracy: 0.9200 - val_loss: 0.1997
Epoch 3/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 620ms/step - accuracy: 0.9084 - loss: 0.2235

88/88 ━━━━━━━━━━━━━━━━━━━━ 69s 787ms/step - accuracy: 0.9084 - loss: 0.2234 - val_accuracy: 0.9243 - val_loss: 0.1863
Epoch 4/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 720ms/step - accuracy: 0.9255 - loss: 0.1864

88/88 ━━━━━━━━━━━━━━━━━━━━ 78s 888ms/step - accuracy: 0.9254 - loss: 0.1866 - val_accuracy: 0.9343 - val_loss: 0.1791
Epoch 5/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 83s 948ms/step - accuracy: 0.9183 - loss: 0.1894 - val_accuracy: 0.9200 - val_loss: 0.1922
Epoch 6/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 675ms/step - accuracy: 0.9259 - loss: 0.1856

88/88 ━━━━━━━━━━━━━━━━━━━━ 73s 831ms/step - accuracy: 0.9259 - loss: 0.1855 - val_accuracy: 0.9243 - val_loss: 0.1672
Epoch 7/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 66s 755ms/step - accuracy: 0.9393 - loss: 0.1609 - val_accuracy: 0.9286 - val_loss: 0.1763
Epoch 8/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 72s 819ms/step - accuracy: 0.9401 - loss: 0.1564 - val_accuracy: 0.9186 - val_loss: 0.1893
Epoch 9/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 67s 767ms/step - accuracy: 0.9277 - loss: 0.1831 - val_accuracy: 0.9286 - val_loss: 0.1784
Model training complete. Best model saved as best_pneumonia_model.h5


In [2]:
#load model
from tensorflow.keras.models import load_model

model = load_model('best_pneumonia_model.h5')

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

val_generator = val_datagen.flow_from_directory(
    'chest_xray/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    color_mode='rgb',
    subset='validation',
    shuffle=False
)

Found 700 images belonging to 2 classes.


In [5]:
#Evaluate the model

loss, accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

c:\Users\ASUS\.conda\envs\GPUT\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


22/22 ━━━━━━━━━━━━━━━━━━━━ 24s 946ms/step - accuracy: 0.9606 - loss: 0.1526
Validation Loss: 0.2186
Validation Accuracy: 0.9229


In [6]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Get true labels and predictions
val_generator.reset()
y_true = val_generator.classes
y_pred_prob = model.predict(val_generator)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Classification report
print(classification_report(y_true, y_pred, target_names=list(val_generator.class_indices.keys())))

# Confusion matrix
print(confusion_matrix(y_true, y_pred))

22/22 ━━━━━━━━━━━━━━━━━━━━ 15s 568ms/step
              precision    recall  f1-score   support

      NORMAL       0.80      1.00      0.89       217
   PNEUMONIA       1.00      0.89      0.94       483

    accuracy                           0.92       700
   macro avg       0.90      0.94      0.92       700
weighted avg       0.94      0.92      0.92       700

[[217   0]
 [ 54 429]]


In [7]:

## Compute Precision, Recall, F1 Score, and Confusion Matrix

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Ensure the generator is not shuffled for correct label order
val_generator.reset()

# Get true labels and predictions
y_true = val_generator.classes
y_pred_prob = model.predict(val_generator)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Print classification report
target_names = list(val_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 516ms/step
              precision    recall  f1-score   support

      NORMAL       0.80      1.00      0.89       217
   PNEUMONIA       1.00      0.89      0.94       483

    accuracy                           0.92       700
   macro avg       0.90      0.94      0.92       700
weighted avg       0.94      0.92      0.92       700

Confusion Matrix:
[[217   0]
 [ 54 429]]
